In [2]:
import os
import requests
import urllib.parse
import datetime
import pandas as pd

In [1]:
def datetime_converter(type, time):
    if type == 'date':
        timestamp = int(time) / 1000
        date = datetime.datetime.fromtimestamp(timestamp)
        return date
    elif type == 'timestamp':
        date = datetime.datetime.strptime(time, '%Y-%m-%d')
        timestamp = int(date.timestamp())
        return timestamp
    else:
        raise ValueError("Invalid type. Supported types are 'date' and 'timestamp'.")


In [3]:
def get_puuid(game_name, name_tag):
    with open('../api_key.txt', 'r') as file:
        api_key = file.read().replace('\n', '')
    url_puuid = f'https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{name_tag}?api_key={api_key}'
    response_puuid = requests.get(url_puuid)
    return response_puuid.json()['puuid']

In [4]:
def get_matchids(puuid, amount_of_games=10000, starttime=0, endtime=0, type='ranked'):
    with open('../api_key.txt', 'r') as file:
        api_key = file.read().replace('\n', '')
    
    match_ids = []
    count = 0
    start = 0
    
    while count < amount_of_games:
        remaining_games = amount_of_games - count
        games_to_fetch = min(remaining_games, 100)
        
        if starttime == 0 and endtime == 0:
            url_matchids = f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type={type}&start={start}&count={games_to_fetch}&api_key={api_key}'
        elif starttime != 0 and endtime == 0:
            url_matchids = f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={starttime}&type={type}&start={start}&count={games_to_fetch}&api_key={api_key}'
        else:
            url_matchids = f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={starttime}&endTime={endtime}&type={type}&start={start}&count={games_to_fetch}&api_key={api_key}'
        
        response_matchids = requests.get(url_matchids)
        match_ids.extend(response_matchids.json())
        
        count += games_to_fetch
        start += games_to_fetch
    
    return match_ids


In [17]:
def get_match_data(match_id):
    try:
        with open('../api_key.txt', 'r') as file:
            api_key = file.read().replace('\n', '')
        url_match = f'https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
        response_match = requests.get(url_match)
        response_json = response_match.json()
        
        if 'status' in response_json and response_json['status']['status_code'] == 404:
            print('Data not found - match file not found')
            return None
        
        return response_json
    except Exception as e:
        print(f'Error occurred: {str(e)}')
        return {}


In [18]:
def filter_data(data, my_puuid, puuids_to_exclude):
    filtered_data = []
    for match in data:
        my_summoner = match['metadata']['participants'].index(my_puuid)
        team_id_my_summoner = match['info']['participants'][my_summoner]['teamId']
        for participant in match['info']['participants']:
            if (participant['teamPosition'] == 'BOTTOM' or participant['teamPosition'] == 'UTILITY') and participant['teamId'] == team_id_my_summoner and participant['puuid'] != puuids_to_exclude:
                filtered_data.append(participant)
    return filtered_data

In [28]:
my_puuid = get_puuid('Zp4t', 'EUW')
puuids_to_exclude = [get_puuid(puuid[0], puuid[1]) for puuid in [('Lurrvpöse', 'EUW'), ('Deadlystream538', 'EUW')]]

match_ids = get_matchids(my_puuid, amount_of_games=1, type='ranked')
filtered_data = []

for match in match_ids:
    match_data = get_match_data(match)
    my_summoner = match_data['metadata']['participants'].index(my_puuid)
    team_id_my_summoner = match_data['info']['participants']

team_id_my_summoner

[{'allInPings': 0,
  'assistMePings': 0,
  'assists': 5,
  'baronKills': 0,
  'basicPings': 0,
  'bountyLevel': 1,
  'challenges': {'12AssistStreakCount': 0,
   'abilityUses': 81,
   'acesBefore15Minutes': 0,
   'alliedJungleMonsterKills': 0,
   'baronTakedowns': 0,
   'blastConeOppositeOpponentCount': 0,
   'bountyGold': 0,
   'buffsStolen': 0,
   'completeSupportQuestInTime': 0,
   'controlWardsPlaced': 0,
   'damagePerMinute': 951.9541098284301,
   'damageTakenOnTeamPercentage': 0.20002106841381037,
   'dancedWithRiftHerald': 0,
   'deathsByEnemyChamps': 7,
   'dodgeSkillShotsSmallWindow': 3,
   'doubleAces': 0,
   'dragonTakedowns': 0,
   'effectiveHealAndShielding': 5177.28955078125,
   'elderDragonKillsWithOpposingSoul': 0,
   'elderDragonMultikills': 0,
   'enemyChampionImmobilizations': 43,
   'enemyJungleMonsterKills': 0,
   'epicMonsterKillsNearEnemyJungler': 0,
   'epicMonsterKillsWithin30SecondsOfSpawn': 0,
   'epicMonsterSteals': 0,
   'epicMonsterStolenWithoutSmite': 0,
 

In [19]:
my_puuid = get_puuid('Zp4t', 'EUW')
puuids_to_exclude = [get_puuid(puuid[0], puuid[1]) for puuid in [('Lurrvpöse', 'EUW'), ('Deadlystream538', 'EUW')]]

match_ids = get_matchids(my_puuid, starttime=datetime_converter('timestamp', '2024-01-04'))
filtered_data = []

for match in match_ids:
    match_data = get_match_data(match)
    if match_data is not None:
        filtered_data.append(filter_data(type(match_data), match_data, my_puuid, puuids_to_exclude))
        break
    else:
        print('Match data not found')
        break

TypeError: string indices must be integers

In [13]:
filtered_data = []

for match in response_matchids.json():
    url_match = f'https://europe.api.riotgames.com/lol/match/v5/matches/{match}?api_key={api_key}'
    response_match = requests.get(url_match)
    match_data = response_match.json()
    my_summoner = match_data['metadata']['participants'].index(puuid)
    team_id_my_summoner = match_data['info']['participants'][my_summoner]['teamId']
    for participant in match_data['info']['participants']:
        if (participant['teamPosition'] == 'BOTTOM' or participant['teamPosition'] == 'UTILITY') and participant['teamId'] == team_id_my_summoner and participant['puuid'] != puuid_lukas:
            filtered_data.append(participant)

df_filtered = pd.DataFrame(filtered_data)


NameError: name 'response_matchids' is not defined

In [20]:
df_filtered.loc[df_filtered['deaths'] == 0, 'deaths'] += 1

df_filtered['kda'] = (df_filtered['kills'] + df_filtered['assists']) / df_filtered['deaths']
df_filtered['kda'].mean()

NameError: name 'df_filtered' is not defined

In [8]:
list = get_matchids(get_puuid('Zp4t', 'EUW'))
len(list)

402

In [9]:
my_puuid = get_puuid('Zp4t', 'EUW')
get_matchids(my_puuid, 50, starttime = datetime_converter('timestamp', '2024-01-30'))

KeyError: 'puuid'

In [12]:
get_match_data(list[1])

{'status': {'message': 'Rate limit exceeded', 'status_code': 429}}

In [ ]:
datetime_converter('date', 1697288568382)

datetime.datetime(2023, 10, 14, 15, 2, 48, 382000)